<a href="https://colab.research.google.com/github/ledomestrinare/awari/blob/master/coletadados_mercadolivre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep
import urllib.request

In [ ]:
zonas = ['sul', 'norte', 'leste', 'oeste']
url_ml = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'
paginas = []
re_quarto ='\| (.*) quarto'
re_area = '> (.*) m²'

In [ ]:
def pega_url(zona):
  zona = zona
  pag = 200
  paginas = []
  numero = 1
  lista_dfs = []
  url_req = ("https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-"+ zona +"/_Desde_"+ str(numero))
  while pag == 200:
    try:
      pag = urllib.request.urlopen(url_req).getcode()
      paginas.append(url_req)
      numero = numero + 48
      url_req = ("https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-"+ zona +"/_Desde_"+ str(numero))
      print(url_req)
    except:
      pag = 404
  return paginas

In [ ]:
def coleta_dados(url):
    conteudo = requests.get(url).content
    sopa = BeautifulSoup(conteudo)
    
    precos = sopa.find_all('span', class_='price__fraction')
    areas_e_quartos = sopa.find_all('div', class_='item__attrs')
    
    lista_areas = []
    lista_quartos = []
    lista_precos = []
    lista_zonas = []
    
    for padrao in precos:
        preco = re.findall('<span class="price__fraction">(.*)</span>', str(padrao))
        lista_precos.append(preco[0].replace('.', ''))
    
    for elemento in areas_e_quartos:
        quarto = re.findall(re_quarto, str(elemento))
        area = re.findall(re_area, str(elemento))
        
        if quarto==[]:
            quarto=np.nan
        else:
            quarto=quarto[0]
        
        if area==[]:
            area=np.nan 
        else:
            area=area[0]

        lista_quartos.append(quarto)
        lista_areas.append(area)
        lista_zonas.append(zona)

    dados = {'zona': lista_zonas,
      'quartos': lista_quartos,
      'area': lista_areas,
      'preco': lista_precos}
  
    df = pd.DataFrame(dados)  
    return df

In [ ]:
lista_dfs = []
for zona in zonas:
  urls = pega_url(zona)
  for pagina in urls:
    #url = url_ml.format(zona, pagina)
    df = coleta_dados(pagina)
    print('Carregando a url', pagina)
    lista_dfs.append(df)
    sleep(2)
print('Dados coletados!')
dados = pd.concat(lista_dfs)
dados

https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_241
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_289
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_337
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_385
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_433
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_481
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_529
https://imov

,zona,quartos,area,preco
0,sul,2,230,1700
1,sul,2,104,1300
2,sul,3,130,1800
3,sul,NaN,100,5000
4,sul,3,80,2300
...,...,...,...,...
43,oeste,3,110,2300
44,oeste,4,300,5000
45,oeste,2,190,4500
46,oeste,03,200,2200


In [ ]:
dados.to_csv("dados_sp.csv")

In [ ]:
dff = pd.read_csv("dados_sp.csv", index_col=0)

In [ ]:
dff

,zona,quartos,area,preco
0,sul,2.0,230.0,1700
1,sul,2.0,104.0,1300
2,sul,3.0,130.0,1800
3,sul,NaN,100.0,5000
4,sul,3.0,80.0,2300
...,...,...,...,...
43,oeste,3.0,110.0,2300
44,oeste,4.0,300.0,5000
45,oeste,2.0,190.0,4500
46,oeste,3.0,200.0,2200
